In [1]:
import numpy as np
import pandas as pd

In [2]:
# run a sample down the tree and populate its word
# alphabet is a list of 2-tuples of function. tuple[0]() returns
# a left element split, tuple[1]() returns a right element split
# depth distance down the tree (root is at depth 1)
# prob_left probability that word will be categorized as zero
# acc accumulated word
# size total depth of the tree
def treePerSample(alphabet, depth, prob_left, acc, size):
    p = np.random.random()
    
    if (depth == size): 
        if (p < prob_left): return (acc,0)
        else: return (acc,1)
    else:
        letter = alphabet[0]
        remain = alphabet[1:]
        if (p<0.5): 
            return treePerSample(remain, depth+1, 
                                 prob_left + (0.5)**(depth+1), 
                                 acc + [letter[0]()], size)
        else:
            return treePerSample(remain, 
                                 depth+1, 
                                 prob_left - (0.5)**(depth+1), 
                                 acc + [letter[1]()], size)

# use treePerSample to generate a decision tree data set
# population number of samples
# size total depth of tree
import string
def generateTreePerSample(population, size):
    ls = []
    def a(alph,n):
        def t(): return (alph + str(n))
        return t
        
    alphabet = [(a('a',n), a('b',n)) 
                for n in range(0, size//2)] 
    
    def randomBelow(cutOff):
        def t(): return np.random.uniform(0, cutOff)
        return t
    def randomAbove(cutOff):
        def t(): return np.random.uniform(cutOff, 2*cutOff)
        return t
    numeric = [(randomBelow(n), randomAbove(n)) 
               for n in range(1, size//2)]
    
    for _ in range(0, population):
           ls.append(treePerSample(alphabet+numeric, 1, 0.5, [], size))
    return ls

# build a data frame for decision tree samples
def main(population, size):
    tree = generateTreePerSample(population, size)
    df = pd.DataFrame([t[0] for t in tree], 
                  columns = ['f' + str(x) for x in range(0, len(tree[0][0]))])
    df["label"] = [t[1] for t in tree] 
    return df

In [3]:
df = main(10000, 10)
df.to_csv("../data/smallRandomCatTree.csv", index=False)

In [4]:
df.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,label
0,b0,b1,a2,b3,b4,0.052854,2.434473,3.478266,1.487640,1
1,a0,b1,a2,b3,a4,0.574877,0.160051,3.426829,5.641662,0
2,b0,a1,a2,a3,b4,0.980134,0.752439,3.729060,3.030402,0
3,a0,b1,b2,b3,b4,0.785974,3.495204,2.229013,2.792165,0
4,b0,a1,b2,a3,a4,0.486634,3.712954,2.377462,5.639110,0


In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.linear_model import LogisticRegression

In [6]:
y = df.label
X = np.array(df.drop(['label'], axis=1))


In [7]:
def to_cat(x):
    if isinstance(x, str):
        if (x[0] == 'a'): 
            return float(0) 
        else: return float(1)
    else: return x

X = np.vectorize(to_cat)(X)

In [8]:
dt = DecisionTreeClassifier(min_samples_leaf=50)

In [9]:
dt.fit(X,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=50,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [10]:
from sklearn.externals.six import StringIO
with open("tree.dot", 'w') as f:
     f = tree.export_graphviz(dt, out_file=f)